# Código principal

Nesse código é implementado todo o pipeline proposto pelo projeto, como definido na imagem a seguir:

![image](../diagramaPI.drawio.png)

## Instalação das bibliotecas necessárias

In [1]:
%pip install numpy
%pip install pandas
%pip install pytesseract
%pip install opencv-python
%pip install matplotlib
%pip install imutils
%pip install ipywidgets
%pip install scikit-learn
%pip install joblib
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     -------------------------------------- 78.3/78.3 kB 482.4 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


## Importações de bibliotecas externas

In [ ]:
import pytesseract
import ipywidgets as widgets
import cv2
import asyncio
import json

Para configurar o `pytesseract` é preciso instalar o seu executável. Se não tiver configurado ele nas variáveis de ambiente, modificar a variável do código abaixo para indicar o caminho até o executável do `tesseract`.

A documentação do `pytesseract` pode ser [acessada aqui](https://pypi.org/project/pytesseract/)

O `tesseract` pode ser instalado [por aqui](https://tesseract-ocr.github.io/tessdoc/Installation.html)

In [ ]:
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

## Importações internas

In [ ]:
from classes.Result import Result

from image_preprocessing.image_preprocessing import preprocess_image
from image_preprocessing.document_digitalization import mark_document_found
from image_preprocessing.rotations import rotate_180

from relevant_info_identification.relevant_info_identification import get_document_info_and_mark_relevant_regions

from image_classification.image_classification import get_rg_probability

from result_selection.result_selection import select_result

## Criando objetos da interface gráfica do projeto

In [ ]:
original_image_widget = widgets.Image(
    format='jpg',
    width=600,
    height=400
)


process_btn = widgets.Button(
    description='Processar',
    button_style='success',
    icon='check'
)

stop_btn = widgets.Button(
    description='Parar',
    button_style='danger',
    icon='times'
)


marked_image_widget = widgets.Image(
    format='jpg',
    width=600,
    height=400
)

infos_json_widget = widgets.Text(
    disabled=True
)

prob_widget = widgets.Text(
    disabled=True
)

pass

## Definindo função do pipeline completo

In [ ]:
original_image = None

def execute_pipeline():
    if original_image is not None:
        # Pré-processamento
        preprocessed_image = preprocess_image(original_image)
        no_rotate_preprocessed_image = preprocessed_image
        rotate_180_preprocessed_image = rotate_180(preprocessed_image)

        # OCR + Algoritmo de identificação
        no_rotate_relevant_info, no_rotate_marked_img = get_document_info_and_mark_relevant_regions(no_rotate_preprocessed_image)
        rotate_180_relevant_info, rotate_180_marked_img = get_document_info_and_mark_relevant_regions(rotate_180_preprocessed_image)

        # Classificador
        no_rotate_rg_prob = get_rg_probability(no_rotate_preprocessed_image)
        rotate_180_rg_prob = get_rg_probability(rotate_180_preprocessed_image)

        # Seleção de resultados
        no_rotate_result = Result(no_rotate_marked_img, no_rotate_relevant_info, no_rotate_rg_prob)
        rotate_180_result = Result(rotate_180_marked_img, rotate_180_relevant_info, rotate_180_rg_prob)
        selected_result = select_result([no_rotate_result, rotate_180_result])
        
        # Mostrando na interface
        marked_image_widget.value = cv2.imencode('.jpg', selected_result.marked_img)[1].tobytes()
        infos_json_widget.value = json.dumps(selected_result.relevant_infos_json)
        prob_widget.value = str(selected_result.rg_probability)


delay_in_seconds = 0.1
camera = cv2.VideoCapture(0)
async def read_image_real_time():
    global original_image
    while True:
        ret, original_image = camera.read()
        frame_copy = original_image.copy()
        marked_image = mark_document_found(frame_copy)
        original_image_widget.value = cv2.imencode('.jpg', marked_image)[1].tobytes()
        await asyncio.sleep(delay_in_seconds)

loop = asyncio.get_event_loop()
task = loop.create_task(read_image_real_time())

def on_stop_btn_click():
    task.cancel()
    camera.release()

stop_btn.on_click(lambda _: on_stop_btn_click())


## Adicionando eventos e criando loop de captura de tela

In [ ]:
delay_in_seconds = 0.1
camera = cv2.VideoCapture(0)
async def read_image_real_time():
    global original_image
    while True:
        ret, original_image = camera.read()
        frame_copy = original_image.copy()
        marked_image = mark_document_found(frame_copy)
        original_image_widget.value = cv2.imencode('.jpg', marked_image)[1].tobytes()
        await asyncio.sleep(delay_in_seconds)

loop = asyncio.get_event_loop()
task = loop.create_task(read_image_real_time())

def on_stop_btn_click():
    task.cancel()
    camera.release()

process_btn.on_click(lambda _: execute_pipeline())
stop_btn.on_click(lambda _: on_stop_btn_click())

## Mostrando interface

In [ ]:
display(original_image_widget)

display(widgets.HBox([
    process_btn,
    stop_btn
]))

display(marked_image_widget)

display(infos_json_widget)

display(prob_widget)